In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1

mps


pip install 'aif360[AdversarialDebiasing]'


In [8]:
# read in data and split
X = torch.load('inputs/bfw_resnet50_face_embeddings.pt').cpu()
y = torch.load('inputs/bfw_resnet50_labels.pt').cpu()
df = pd.read_csv('inputs/bfw_resnet50_df.csv')
gender = df['reference_gender']
ethnicity = df['reference_ethnicity']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
0,n004721,n004721,asian,asian,male,male,1.0
1,n004721,n004721,asian,asian,male,male,1.0
2,n004721,n004721,asian,asian,male,male,1.0
3,n004721,n004721,asian,asian,male,male,1.0
4,n004721,n004721,asian,asian,male,male,1.0
...,...,...,...,...,...,...,...
38395,n003412,n005685,white,white,male,male,0.0
38396,n008917,n008839,black,black,female,female,0.0
38397,n008452,n008150,asian,asian,male,male,0.0
38398,n004936,n002509,asian,asian,female,female,0.0


In [10]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]


train_gender = gender[train_split].values
train_gender[train_gender=='male'] = 0
train_gender[train_gender=='female'] = 1
train_gender = train_gender.astype(int)

train_ethnicity = ethnicity[train_split].values

gender_val = gender[val_split].values
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)

ethnicity_val = ethnicity[val_split].values

test_gender = gender[test_split].values
test_gender[test_gender=='male'] = 0
test_gender[test_gender=='female'] = 1
test_gender = test_gender.astype(int)

test_ethnicity = ethnicity[test_split].values

In [4]:
train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_gender'].value_counts())
print('matches')
print(train_matches['reference_gender'].value_counts())

female_non_matches = train_non_matches[train_non_matches['reference_gender']=='female']
male_non_matches = train_non_matches[train_non_matches['reference_gender']=='male']

female_matches = train_matches[train_matches['reference_gender']=='female']
male_matches = train_matches[train_matches['reference_gender']=='male']

non matches
female    5804
male      5675
Name: reference_gender, dtype: int64
matches
female    5809
male      5752
Name: reference_gender, dtype: int64


In [11]:
np.random.seed(random_state)
female_matches_sub_idx = female_matches.index[np.random.choice(len(female_matches.index), size=5675, replace=False)]
male_matches_sub_idx = male_matches.index[np.random.choice(len(male_matches.index), size=5675, replace=False)]
female_non_matches_sub_idx = female_non_matches.index[np.random.choice(len(female_non_matches.index), size=5675, replace=False)]
male_non_matches_sub_idx = male_non_matches.index

X_train = torch.cat([X[female_matches_sub_idx],X[male_matches_sub_idx],X[female_non_matches_sub_idx],X[male_non_matches_sub_idx]])
y_train = torch.cat([y[female_matches_sub_idx],y[male_matches_sub_idx],y[female_non_matches_sub_idx],y[male_non_matches_sub_idx]])
gender_train = np.concatenate([gender[female_matches_sub_idx],gender[male_matches_sub_idx],gender[female_non_matches_sub_idx],gender[male_non_matches_sub_idx]])
ethnicity_train = np.concatenate([ethnicity[female_matches_sub_idx],ethnicity[male_matches_sub_idx],ethnicity[female_non_matches_sub_idx],ethnicity[male_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[female_matches_sub_idx],df.iloc[male_matches_sub_idx],df.iloc[female_non_matches_sub_idx],df.iloc[male_non_matches_sub_idx]])
df_train

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
17999,n000382,n000382,white,white,female,female,1.0
10563,n006634,n006634,asian,asian,female,female,1.0
17735,n001010,n001010,white,white,female,female,1.0
12546,n000460,n000460,black,black,female,female,1.0
12766,n003694,n003694,black,black,female,female,1.0
...,...,...,...,...,...,...,...
24318,n003465,n001976,asian,asian,male,male,0.0
28359,n004960,n009152,black,black,male,male,0.0
26106,n000964,n000512,white,white,male,male,0.0
26500,n009032,n005807,white,white,male,male,0.0


In [12]:
# test_split, holdout_split = train_test_split(np.arange(7680),test_size=0.1, random_state=random_state)
X_test = test_X#[test_split]
# X_holdout = test_X[holdout_split]
y_test = test_y#[test_split]
# y_holdout = test_y[holdout_split]
gender_test = test_gender#[test_split]
# gender_holdout = test_gender[holdout_split]
df_test = test_df#.iloc[test_split]
# df_holdout = test_df.iloc[holdout_split]
ethnicity_test = test_ethnicity#[test_split]
df_test

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
340,n005178,n005178,asian,asian,male,male,1.0
18908,n001229,n001229,white,white,female,female,1.0
37816,n008972,n003463,asian,asian,male,male,0.0
10524,n000138,n000138,asian,asian,female,female,1.0
7492,n003398,n003398,white,white,male,male,1.0
...,...,...,...,...,...,...,...
7045,n007184,n007184,indian,indian,male,male,1.0
10213,n004472,n004472,asian,asian,female,female,1.0
12000,n002274,n002274,black,black,female,female,1.0
36114,n000660,n008636,white,white,female,female,0.0


In [13]:
val_matches = val_df[val_df.labels==1]
val_non_matches = val_df[val_df.labels==0]
print('non matches')
print(val_non_matches['reference_gender'].value_counts())
print('matches')
print(val_matches['reference_gender'].value_counts())

val_female_non_matches = val_non_matches[val_non_matches['reference_gender']=='female']
val_male_non_matches = val_non_matches[val_non_matches['reference_gender']=='male']

val_female_matches = val_matches[val_matches['reference_gender']=='female']
val_male_matches = val_matches[val_matches['reference_gender']=='male']

non matches
female    1996
male      1883
Name: reference_gender, dtype: int64
matches
male      1925
female    1876
Name: reference_gender, dtype: int64


In [14]:
test_matches = df_test[df_test.labels==1]
test_non_matches = df_test[df_test.labels==0]
print('non matches')
print(test_non_matches['reference_gender'].value_counts())
print('matches')
print(test_matches['reference_gender'].value_counts())

test_female_non_matches = test_non_matches[test_non_matches['reference_gender']=='female']
test_male_non_matches = test_non_matches[test_non_matches['reference_gender']=='male']

test_female_matches = test_matches[test_matches['reference_gender']=='female']
test_male_matches = test_matches[test_matches['reference_gender']=='male']

non matches
male      1926
female    1916
Name: reference_gender, dtype: int64
matches
male      1923
female    1915
Name: reference_gender, dtype: int64


In [15]:
# holdout_matches = df_holdout[df_holdout.labels==1]
# holdout_non_matches = df_holdout[df_holdout.labels==0]
# print('non matches')
# print(holdout_non_matches['reference_gender'].value_counts())
# print('matches')
# print(holdout_matches['reference_gender'].value_counts())

# holdout_female_non_matches = holdout_non_matches[holdout_non_matches['reference_gender']=='female']
# holdout_male_non_matches = holdout_non_matches[holdout_non_matches['reference_gender']=='male']

# holdout_female_matches = holdout_matches[holdout_matches['reference_gender']=='female']
# holdout_male_matches = holdout_matches[holdout_matches['reference_gender']=='male']

In [16]:
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)


    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2

In [17]:
eer_thresh = 0.4337
val_cos = cos_sim(X_val[:,:2048],X_val[:,2048:])


eer_pred = (val_cos>eer_thresh)*1

df_bias = val_df.drop(['reference_identity','candidate_identity','reference_ethnicity','candidate_ethnicity','candidate_gender'],axis=1)
df_bias['eer_pred'] = eer_pred


df_bias_female = df_bias[df_bias['reference_gender']=='female']
df_bias_male = df_bias[df_bias['reference_gender']=='male']

female_eer_tn, female_eer_fp, female_eer_fn, female_eer_tp, female_eer_acc = confusion_mat(df_bias_female['eer_pred'], df_bias_female['labels'])
male_eer_tn, male_eer_fp, male_eer_fn, male_eer_tp, male_eer_acc = confusion_mat(df_bias_male['eer_pred'], df_bias_male['labels'])



female_eer_tnr = female_eer_tn/(female_eer_tn+female_eer_fp)
female_eer_tpr = female_eer_tp/(female_eer_tp+female_eer_fn)
female_eer_fnr = female_eer_fn/(female_eer_fn+female_eer_tp)
female_eer_fpr = female_eer_fp/(female_eer_tn+female_eer_fp)

male_eer_tnr = male_eer_tn/(male_eer_tn+male_eer_fp)
male_eer_tpr = male_eer_tp/(male_eer_tp+male_eer_fn)
male_eer_fnr = male_eer_fn/(male_eer_fn+male_eer_tp)
male_eer_fpr = male_eer_fp/(male_eer_tn+male_eer_fp)

print(female_eer_fnr/male_eer_fnr)
print(female_eer_fpr/male_eer_fpr)
print(male_eer_tnr/female_eer_tnr)
print(male_eer_tpr/female_eer_tpr)
print((1-female_eer_acc)/(1-male_eer_acc))
print('female fnr:',female_eer_fnr,'male fnr:',male_eer_fnr)
print('female fpr:',female_eer_fpr,'male fpr:',male_eer_fpr)
print('male tnr:',male_eer_tnr,'female tnr:',female_eer_tnr)
print('male tpr:',male_eer_tpr,'female tpr:',female_eer_tpr)
print('female mis class:',(1-female_eer_acc),'male mis class:',(1-male_eer_acc))
print(male_eer_acc,female_eer_acc)

1.0394456289978677
1.0939272161343965
1.004959725274617
1.0016462736373748
1.0754917597022882
female fnr: 0.04157782515991471 male fnr: 0.04
female fpr: 0.05460921843687375 male fpr: 0.04992033988316516
male tnr: 0.9500796601168349 female tnr: 0.9453907815631263
male tpr: 0.96 female tpr: 0.9584221748400853
female mis class: 0.048295454545454586 male mis class: 0.0449054621848739
0.9550945378151261 0.9517045454545454


In [20]:
white = ethnicity_val == 'white'
black = ethnicity_val == 'black'
asian = ethnicity_val == 'asian'
indian = ethnicity_val == 'indian'

In [36]:
eer_thresh = 0.4337
val_cos = cos_sim(X_val[white,:2048],X_val[white,2048:])


eer_pred = (val_cos>eer_thresh)*1

df_bias = val_df.drop(['reference_identity','candidate_identity','reference_ethnicity','candidate_ethnicity','candidate_gender'],axis=1).iloc[white]
df_bias['eer_pred'] = eer_pred

df
df_bias_female = df_bias[df_bias['reference_gender']=='female']
df_bias_male = df_bias[df_bias['reference_gender']=='male']

female_eer_tn, female_eer_fp, female_eer_fn, female_eer_tp, female_eer_acc = confusion_mat(df_bias_female['eer_pred'], df_bias_female['labels'])
male_eer_tn, male_eer_fp, male_eer_fn, male_eer_tp, male_eer_acc = confusion_mat(df_bias_male['eer_pred'], df_bias_male['labels'])



female_eer_tnr = female_eer_tn/(female_eer_tn+female_eer_fp)
female_eer_tpr = female_eer_tp/(female_eer_tp+female_eer_fn)
female_eer_fnr = female_eer_fn/(female_eer_fn+female_eer_tp)
female_eer_fpr = female_eer_fp/(female_eer_tn+female_eer_fp)

male_eer_tnr = male_eer_tn/(male_eer_tn+male_eer_fp)
male_eer_tpr = male_eer_tp/(male_eer_tp+male_eer_fn)
male_eer_fnr = male_eer_fn/(male_eer_fn+male_eer_tp)
male_eer_fpr = male_eer_fp/(male_eer_tn+male_eer_fp)

print(female_eer_fnr/male_eer_fnr)
print(female_eer_fpr/male_eer_fpr)
print(male_eer_tnr/female_eer_tnr)
print(male_eer_tpr/female_eer_tpr)
print((1-female_eer_acc)/(1-male_eer_acc))
print('female fnr:',female_eer_fnr,'male fnr:',male_eer_fnr)
print('female fpr:',female_eer_fpr,'male fpr:',male_eer_fpr)
print('male tnr:',male_eer_tnr,'female tnr:',female_eer_tnr)
print('male tpr:',male_eer_tpr,'female tpr:',female_eer_tpr)
print('female mis class:',(1-female_eer_acc),'male mis class:',(1-male_eer_acc))
print(male_eer_acc,female_eer_acc)

2.5107991360691146
1.0988372093023255
1.001236993378447
1.0201468894009216
1.7808988764044942
female fnr: 0.032397408207343416 male fnr: 0.012903225806451613
female fpr: 0.013565891472868217 male fpr: 0.012345679012345678
male tnr: 0.9876543209876543 female tnr: 0.9864341085271318
male tpr: 0.9870967741935484 female tpr: 0.9676025917926566
female mis class: 0.022471910112359605 male mis class: 0.012618296529968487
0.9873817034700315 0.9775280898876404


In [12]:
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)
gender_val
val_bias_df = pd.DataFrame(columns=['label','prediction','protected','rounded'])
val_bias_df['label']=y_val.cpu().numpy()
val_bias_df['prediction']=val_cos.cpu().numpy()
val_bias_df['rounded']=eer_pred.cpu().numpy()
val_bias_df['protected']=gender_val
unpriveleged_df = val_bias_df[val_bias_df['protected']==1]
priveleged_df = val_bias_df[val_bias_df['protected']==0]

In [13]:
unpriveleged_tn, unpriveleged_fp, unpriveleged_fn, unpriveleged_tp, unpriveleged_acc = confusion_mat(unpriveleged_df['rounded'], unpriveleged_df['label'])
priveleged_tn, priveleged_fp, priveleged_fn, priveleged_tp, priveleged_acc = confusion_mat(priveleged_df['rounded'], priveleged_df['label'])
unpriveleged_fnr = unpriveleged_fn/(unpriveleged_fn+unpriveleged_tp)
unpriveleged_fpr = unpriveleged_fp/(unpriveleged_fp+unpriveleged_tn)
priveleged_fnr = priveleged_fn/(priveleged_fn+priveleged_tp)
priveleged_fpr = priveleged_fp/(priveleged_fp+priveleged_tn)

print('Overall Accuracy:',round(torch.sum(eer_pred == y_val).item()/y_val.size()[0],3))
print('Missclassificaition Rate Ratio:',round((1-unpriveleged_acc)/(1- priveleged_acc),3))
print('FNR Ratio:',round(unpriveleged_fnr/priveleged_fnr,3),'FPR Ratio:',round(unpriveleged_fpr/priveleged_fpr,3))
print('AOE:',round(AOE(unpriveleged_tn,unpriveleged_fp,unpriveleged_fn,unpriveleged_tp,priveleged_tn,priveleged_fp,priveleged_fn,priveleged_tp),3))

Overall Accuracy: 0.954
Missclassificaition Rate Ratio: 1.07
FNR Ratio: 1.039 FPR Ratio: 1.084
AOE: 0.003


In [14]:
def to_dataframe(y_true, y_pred, y_prot):
        y_true, y_pred, y_prot = y_true.float().cpu().numpy(), y_pred.float().cpu().numpy(), y_prot.astype(np.float32)
        df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_prot': y_prot})
        dataset = StandardDataset(df, 'y_true', [1.], ['y_prot'], [[1.]])
        dataset.scores = y_pred.reshape(-1, 1)
        return dataset

In [15]:
aif_data = to_dataframe(y_val, val_cos, gender_val)

In [16]:
eo = EqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                  unprivileged_groups=[{'y_prot': 1.}],seed=random_state)

eo=eo.fit(aif_data,aif_data)

eo_pred = eo.predict(aif_data).labels.reshape(-1)
eo_label = aif_data.labels.reshape(-1)
eo_prot = aif_data.protected_attributes.reshape(-1)

eo_unpriv_ids = (eo_prot==1)
eo_unpriv_pred = eo_pred[eo_unpriv_ids]
eo_unpriv_label = eo_label[eo_unpriv_ids]

eo_priv_ids = (eo_prot==0)
eo_priv_pred = eo_pred[eo_priv_ids]
eo_priv_label = eo_label[eo_priv_ids]

eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp, eo_unpriv_acc = confusion_mat(eo_unpriv_pred, eo_unpriv_label)
eo_unpriv_fpr = eo_unpriv_fp/(eo_unpriv_fp+eo_unpriv_tn)
eo_unpriv_fnr = eo_unpriv_fn/(eo_unpriv_fn+eo_unpriv_tp)

eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp, eo_priv_acc = confusion_mat(eo_priv_pred, eo_priv_label)
eo_priv_fpr = eo_priv_fp/(eo_priv_fp+eo_priv_tn)
eo_priv_fnr = eo_priv_fn/(eo_priv_fn+eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-eo_unpriv_acc)/(1- eo_priv_acc),3))
print('FPR Ratio:',eo_unpriv_fpr/eo_priv_fpr)
print('FNR Ratio:',eo_unpriv_fnr/eo_priv_fnr)
print(np.sum(eo_pred == eo_label)/len(eo_pred))
print('AOE:',round(AOE(eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp,eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp),3))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [17]:
gender_test[gender_test=='male'] = 0
gender_test[gender_test=='female'] = 1
gender_test = gender_test.astype(int)
gender_test
test_bias_df = pd.DataFrame(columns=['label','prediction','protected','rounded'])
test_bias_df['label']=y_test.cpu().numpy()
test_cos = cos_sim(X_test[:,:2048],X_test[:,2048:])
test_eer_pred = (test_cos>eer_thresh)*1
test_bias_df['prediction']=test_cos.cpu().numpy()
test_bias_df['rounded']=test_eer_pred.cpu().numpy()
test_bias_df['protected']=gender_test

test_unpriveleged_df = test_bias_df[test_bias_df['protected']==1]

test_priveleged_df = test_bias_df[test_bias_df['protected']==0]

test_aif_data = to_dataframe(y_test, test_cos, gender_test)

test_eo_pred = eo.predict(test_aif_data).labels.reshape(-1)
test_eo_label = test_aif_data.labels.reshape(-1)
test_eo_prot = test_aif_data.protected_attributes.reshape(-1)

test_eo_unpriv_ids = (test_eo_prot==1)
test_eo_unpriv_pred = test_eo_pred[test_eo_unpriv_ids]
test_eo_unpriv_label = test_eo_label[test_eo_unpriv_ids]

test_eo_priv_ids = (test_eo_prot==0)
test_eo_priv_pred = test_eo_pred[test_eo_priv_ids]
test_eo_priv_label = test_eo_label[test_eo_priv_ids]

test_eo_unpriv_tn, test_eo_unpriv_fp, test_eo_unpriv_fn, test_eo_unpriv_tp, test_eo_unpriv_acc = confusion_mat(test_eo_unpriv_pred, test_eo_unpriv_label)
test_eo_unpriv_fpr = test_eo_unpriv_fp/(test_eo_unpriv_fp+test_eo_unpriv_tn)
test_eo_unpriv_fnr = test_eo_unpriv_fn/(test_eo_unpriv_fn+test_eo_unpriv_tp)

test_eo_priv_tn, test_eo_priv_fp, test_eo_priv_fn, test_eo_priv_tp, test_eo_priv_acc = confusion_mat(test_eo_priv_pred, test_eo_priv_label)
test_eo_priv_fpr = test_eo_priv_fp/(test_eo_priv_fp+test_eo_priv_tn)
test_eo_priv_fnr = test_eo_priv_fn/(test_eo_priv_fn+test_eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-test_eo_unpriv_acc)/(1- test_eo_priv_acc),3))
print('FPR Ratio:',test_eo_unpriv_fpr/test_eo_priv_fpr)
print('FNR Ratio:',test_eo_unpriv_fnr/test_eo_priv_fnr)
print(np.sum(test_eo_pred == test_eo_label)/len(test_eo_pred))
print('AOE:',round(AOE(test_eo_unpriv_tn, test_eo_unpriv_fp, test_eo_unpriv_fn, test_eo_unpriv_tp,test_eo_priv_tn, test_eo_priv_fp, test_eo_priv_fn, test_eo_priv_tp),3))

Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [18]:
cost_constraint = 'weighted'
privileged_groups=[{'y_prot': 0.}],
unprivileged_groups=[{'y_prot': 1.}]
cpp = CalibratedEqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                             unprivileged_groups=[{'y_prot': 1.}],
                                             cost_constraint=cost_constraint,seed=random_state)
cpp=cpp.fit(aif_data,aif_data)
cpp_pred = cpp.predict(aif_data).labels.reshape(-1)
cpp_label = aif_data.labels.reshape(-1)
cpp_prot = aif_data.protected_attributes.reshape(-1)

unpriv_ids = (cpp_prot==1)
unpriv_pred = cpp_pred[unpriv_ids]
unpriv_label = cpp_label[unpriv_ids]

priv_ids = (cpp_prot==0)
priv_pred = cpp_pred[priv_ids]
priv_label = cpp_label[priv_ids]

unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp, unpriv_acc = confusion_mat(unpriv_pred, unpriv_label)
unpriv_fpr = unpriv_fp/(unpriv_fp+unpriv_tn)
unpriv_fnr = unpriv_fn/(unpriv_fn+unpriv_tp)

priv_tn, priv_fp, priv_fn, priv_tp, priv_acc = confusion_mat(priv_pred, priv_label)
priv_fpr = priv_fp/(priv_fp+priv_tn)
priv_fnr = priv_fn/(priv_fn+priv_tp)

print('Missclassificaition Rate Ratio:',round((1-unpriv_acc)/(1- priv_acc),3))
print('FPR Ratio:',round(unpriv_fpr/priv_fpr,3))
print('FNR Ratio:',round(unpriv_fnr/priv_fnr,3))
print('Overall Accuracy:',round(np.sum(cpp_pred == cpp_label)/len(cpp_label),3))
print('AOE:',round(AOE(unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp,priv_tn, priv_fp, priv_fn, priv_tp),3))

Missclassificaition Rate Ratio: 0.623
FPR Ratio: 0.081
FNR Ratio: 1.237
Overall Accuracy: 0.932
AOE: 0.049


In [19]:
test_cpp_pred = cpp.predict(test_aif_data).labels.reshape(-1)
test_cpp_label = test_aif_data.labels.reshape(-1)
test_cpp_prot = test_aif_data.protected_attributes.reshape(-1)

test_cpp_unpriv_ids = (test_cpp_prot==1)
test_cpp_unpriv_pred = test_cpp_pred[test_cpp_unpriv_ids]
test_cpp_unpriv_label = test_cpp_label[test_cpp_unpriv_ids]

test_cpp_priv_ids = (test_cpp_prot==0)
test_cpp_priv_pred = test_cpp_pred[test_cpp_priv_ids]
test_cpp_priv_label = test_cpp_label[test_cpp_priv_ids]

test_cpp_unpriv_tn, test_cpp_unpriv_fp, test_cpp_unpriv_fn, test_cpp_unpriv_tp, test_cpp_unpriv_acc = confusion_mat(test_cpp_unpriv_pred, test_cpp_unpriv_label)
test_cpp_unpriv_fpr = test_cpp_unpriv_fp/(test_cpp_unpriv_fp+test_cpp_unpriv_tn)
test_cpp_unpriv_fnr = test_cpp_unpriv_fn/(test_cpp_unpriv_fn+test_cpp_unpriv_tp)

test_cpp_priv_tn, test_cpp_priv_fp, test_cpp_priv_fn, test_cpp_priv_tp, test_cpp_priv_acc = confusion_mat(test_cpp_priv_pred, test_cpp_priv_label)
test_cpp_priv_fpr = test_cpp_priv_fp/(test_cpp_priv_fp+test_cpp_priv_tn)
test_cpp_priv_fnr = test_cpp_priv_fn/(test_cpp_priv_fn+test_cpp_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-test_cpp_unpriv_acc)/(1- test_cpp_priv_acc),3))
print('FPR Ratio:',test_cpp_unpriv_fpr/test_cpp_priv_fpr)
print('FNR Ratio:',test_cpp_unpriv_fnr/test_cpp_priv_fnr)
print(np.sum(test_cpp_pred == test_cpp_label)/len(test_cpp_pred))
print('AOE:',round(AOE(test_cpp_unpriv_tn, test_cpp_unpriv_fp, test_cpp_unpriv_fn, test_cpp_unpriv_tp,test_cpp_priv_tn, test_cpp_priv_fp, test_cpp_priv_fn, test_cpp_priv_tp),3))

Missclassificaition Rate Ratio: 0.636
FPR Ratio: 0.15564684490538083
FNR Ratio: 1.1223160804791892
0.9345052083333333
AOE: 0.039


In [20]:
roc = RejectOptionClassification(unprivileged_groups=[{'y_prot': 1.}],
                                         privileged_groups=[{'y_prot': 0.}],
                                         low_class_thresh=0.01, high_class_thresh=0.99,
                                         num_class_thresh=100, num_ROC_margin=50,
                                         metric_name="Average odds difference",
                                         metric_ub=0.05, metric_lb=-0.05)

roc = roc.fit(aif_data, aif_data)

roc_pred = roc.predict(aif_data).labels.reshape(-1)
roc_label = aif_data.labels.reshape(-1)
roc_prot = aif_data.protected_attributes.reshape(-1)

roc_unpriv_ids = (roc_prot==1)
roc_unpriv_pred = roc_pred[roc_unpriv_ids]
roc_unpriv_label = roc_label[roc_unpriv_ids]

roc_priv_ids = (roc_prot==0)
roc_priv_pred = roc_pred[roc_priv_ids]
roc_priv_label = roc_label[roc_priv_ids]

roc_unpriv_tn, roc_unpriv_fp, roc_unpriv_fn, roc_unpriv_tp, roc_unpriv_acc = confusion_mat(roc_unpriv_pred, roc_unpriv_label)
roc_unpriv_fpr = roc_unpriv_fp/(roc_unpriv_fp+roc_unpriv_tn)
roc_unpriv_fnr = roc_unpriv_fn/(roc_unpriv_fn+roc_unpriv_tp)

roc_priv_tn, roc_priv_fp, roc_priv_fn, roc_priv_tp, roc_priv_acc = confusion_mat(roc_priv_pred, roc_priv_label)
roc_priv_fpr = roc_priv_fp/(roc_priv_fp+roc_priv_tn)
roc_priv_fnr = roc_priv_fn/(roc_priv_fn+roc_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-roc_unpriv_acc)/(1- roc_priv_acc),3))
print('FPR Ratio:',round(roc_unpriv_fpr/roc_priv_fpr,3))
print('FNR Ratio:',round(roc_unpriv_fnr/roc_priv_fnr,3))
print(np.sum(roc_pred == roc_label)/len(roc_pred))
print('AOE:',round(AOE(roc_unpriv_tn, roc_unpriv_fp, roc_unpriv_fn, roc_unpriv_tp,roc_priv_tn, roc_priv_fp, roc_priv_fn, roc_priv_tp),3))


Missclassificaition Rate Ratio: 1.065
FPR Ratio: 1.029
FNR Ratio: 1.11
0.9576822916666666
AOE: 0.004


In [21]:
test_roc_pred = roc.predict(test_aif_data).labels.reshape(-1)
test_roc_label = test_aif_data.labels.reshape(-1)
test_roc_prot = test_aif_data.protected_attributes.reshape(-1)

test_roc_unpriv_ids = (test_roc_prot==1)
test_roc_unpriv_pred = test_roc_pred[test_roc_unpriv_ids]
test_roc_unpriv_label = test_roc_label[test_roc_unpriv_ids]

test_roc_priv_ids = (test_roc_prot==0)
test_roc_priv_pred = test_roc_pred[test_roc_priv_ids]
test_roc_priv_label = test_roc_label[test_roc_priv_ids]

test_roc_unpriv_tn, test_roc_unpriv_fp, test_roc_unpriv_fn, test_roc_unpriv_tp, test_roc_unpriv_acc = confusion_mat(test_roc_unpriv_pred, test_roc_unpriv_label)
test_roc_unpriv_fpr = test_roc_unpriv_fp/(test_roc_unpriv_fp+test_roc_unpriv_tn)
test_roc_unpriv_fnr = test_roc_unpriv_fn/(test_roc_unpriv_fn+test_roc_unpriv_tp)

test_roc_priv_tn, test_roc_priv_fp, test_roc_priv_fn, test_roc_priv_tp, test_roc_priv_acc = confusion_mat(test_roc_priv_pred, test_roc_priv_label)
test_roc_priv_fpr = test_roc_priv_fp/(test_roc_priv_fp+test_roc_priv_tn)
test_roc_priv_fnr = test_roc_priv_fn/(test_roc_priv_fn+test_roc_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-test_roc_unpriv_acc)/(1- test_roc_priv_acc),3))
print('FPR Ratio:',test_roc_unpriv_fpr/test_roc_priv_fpr)
print('FNR Ratio:',test_roc_unpriv_fnr/test_roc_priv_fnr)
print(np.sum(test_roc_pred == test_roc_label)/len(test_roc_pred))
print('AOE:',round(AOE(test_roc_unpriv_tn, test_roc_unpriv_fp, test_roc_unpriv_fn, test_roc_unpriv_tp,test_roc_priv_tn, test_roc_priv_fp, test_roc_priv_fn, test_roc_priv_tp),3))

Missclassificaition Rate Ratio: 1.201
FPR Ratio: 1.2697505768596857
FNR Ratio: 1.1784562932913276
0.9545572916666667
AOE: 0.008
